<p style="padding: 10px; border: 1px solid black;">
<img src="../mlu_utils/MLU-NEW-logo.png" alt="drawing" width="400"/> <br/>

# <a name="0">Machine Learning University</a>
## <a name="0">Lab 3: Prompt Engineering</a>

Prompt engineering is the process of constructing and refining input prompts to improve the quality of generated responses from language models. It's an iterative process that requires experimentation to find the optimal approach for a given problem.

Key strategies for effective prompt engineering include:
- Writing clear and specific instructions
- Highlighting or specifying the relevant parts of the prompt
- Adding details or restrictions to guide the model's output
- Instructing the model to follow a step-by-step approach for complex tasks

In this lab, we'll use the Amazon Titan Text G1 - Premier model to demonstrate various prompt engineering techniques across different machine learning tasks.
    
1. <a href="#1">Installing libraries</a>
2. <a href="#2">Example problems</a>
    1. <a href="#2.1">Text summarization</a>
    2. <a href="#2.2">Question-answering</a>
    3. <a href="#2.3">Text generation</a>
    4. <a href="#2.4">In-context learning</a>
    5. <a href="#2.5">Chain-of-thought</a>
3. <a href="#3">Quizzes</a>

### In this lab, we will cover topics such as:
- Introduction to Prompt Engineering
- Text Summarization
- Question Answering 
- Text Generation
- In-context Learning: Zero-shot, One-shot, and Few-shot Learning
- Chain of Thought
    
Let's begin by setting up our environment and connecting to the Bedrock service.

---

### <a name="1">Installation of libraries </a>
(<a href="#0">Go to top</a>)

We first install recent versions of boto3 and botocore.

In [ ]:
%%capture
!pip install -q -r ../requirements.txt

We access the Bedrock service through boto3 by providing the service name, region name and endpoint URL.

In [ ]:
import json, boto3

session = boto3.session.Session()

bedrock_inference = session.client(
    service_name="bedrock-runtime",
    region_name=session.region_name,
)

Let's specify the API parameters. We will use the __Amazon Titan Text G1 - Premier__ model.

In [ ]:
def send_prompt(prompt_data, temperature=0.0, top_p=0.5, max_token_count=1000):

    body = json.dumps(
        {
            "inputText": prompt_data,
            "textGenerationConfig": {
                "temperature": temperature,
                "topP": top_p,
                "maxTokenCount": max_token_count,
            },
        }
    )

    modelId = "amazon.titan-text-premier-v1:0"

    accept = "application/json"
    contentType = "application/json"

    response = bedrock_inference.invoke_model(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )

    response_body = json.loads(response["body"].read())

    return response_body["results"][0]["outputText"]

## <a name="2">Example problems</a>
(<a href="#0">Go to top</a>)

### <a name="2.1">1) Text summarization:</a>
(<a href="#0">Go to top</a>)

With text summarization, the main purpose is to create a shorter version of a given text while preserving the relevant information in it. 

We use the following text from the [sustainability section](https://sustainability.aboutamazon.com/environment/renewable-energy) of [about.amazon.com](https://www.aboutamazon.com/).


<p style="font-size:12pt;">
    "In 2021, we reached 85% renewable energy across our business. Our first solar projects in South Africa and the United Arab Emirates came online, and we announced new projects in Singapore, Japan, Australia, and China. Our projects in South Africa and Japan are the first corporate-backed, utility-scale solar farms in these countries. We also announced two new offshore wind projects in Europe, including our largest renewable energy project to date. As of December 2021, we had enabled more than 3.5 gigawatts of renewable energy in Europe through 80 projects, making Amazon the largest purchaser of renewable energy in Europe."
</p>

Let's start with the first summarization example below. We pass this text as well as the instruction to summarize it. The instruction part of the prompt becomes __The following is a text about Amazon. Summarize this:__

In [ ]:
prompt_data = """The following is a text about Amazon. Summarize this:  \
Text: In 2021, we reached 85% renewable energy across our business.\
Our first solar projects in South Africa and the United Arab Emirates\
came online, and we announced new projects in Singapore, Japan, \
Australia, and China. Our projects in South Africa and Japan are \
the first corporate-backed, utility-scale solar farms in these \
countries. We also announced two new offshore wind projects in \
Europe, including our largest renewable energy project to date.\
As of December 2021, we had enabled more than 3.5 gigawatts of \
renewable energy in Europe through 80 projects, making Amazon \
the largest purchaser of renewable energy in Europe."""

print(send_prompt(prompt_data, temperature=0.0))

Nice. This text is shorter. We can set the desired lenght of the summary by adding more constraints to the instructions. Let's create a one-sentence summary of this text. The instruction part of the prompt becomes the following: __The following is a text about Amazon. Summarize it in one sentence.__

In [ ]:
prompt_data = """The following is a text about Amazon. Summarize it in one sentence. \
Text: In 2021, we reached 85% renewable energy across our business.\
Our first solar projects in South Africa and the United Arab Emirates\
came online, and we announced new projects in Singapore, Japan, \
Australia, and China. Our projects in South Africa and Japan are \
the first corporate-backed, utility-scale solar farms in these \
countries. We also announced two new offshore wind projects in \
Europe, including our largest renewable energy project to date.\
As of December 2021, we had enabled more than 3.5 gigawatts of \
renewable energy in Europe through 80 projects, making Amazon \
the largest purchaser of renewable energy in Europe."""

print(send_prompt(prompt_data, temperature=0.0))

Nice! The model generated a one-sentence summary.

---

### <a name="2.2">2) Question Answering</a>
(<a href="#0">Go to top</a>)

In Question Answering problem, a Machine Learning model answers some questions using some provided context. Here as context, we will use the previous text about Amazon's sustainability efforts. 

The first question is asking about the names of the countries mentioned in the text. 

The instruction section of the prompt is __What are the names of the countries in the following text?__

In [ ]:
prompt_data = """What are the names of the countries in the following text? \
Text: In 2021, we reached 85% renewable energy across our business.\
Our first solar projects in South Africa and the United Arab Emirates\
came online, and we announced new projects in Singapore, Japan, \
Australia, and China. Our projects in South Africa and Japan are \
the first corporate-backed, utility-scale solar farms in these \
countries. We also announced two new offshore wind projects in \
Europe, including our largest renewable energy project to date.\
As of December 2021, we had enabled more than 3.5 gigawatts of \
renewable energy in Europe through 80 projects, making Amazon \
the largest purchaser of renewable energy in Europe."""

print(send_prompt(prompt_data, temperature=0.0))

Nice. We get all of the geographical places mentioned in the text. 

Let's try to learn something specific about the document. For example, the amount of gigawatts that the project in Europe enabled. 

The instruction section of the prompt is __How many gigawatts of energy did Amazon enable in Europe according to the following text?__

In [ ]:
prompt_data = """How many gigawatts of energy did Amazon enable in \
Europe according to the following text? \
Text: In 2021, we reached 85% renewable energy across our business.\
Our first solar projects in South Africa and the United Arab Emirates\
came online, and we announced new projects in Singapore, Japan, \
Australia, and China. Our projects in South Africa and Japan are \
the first corporate-backed, utility-scale solar farms in these \
countries. We also announced two new offshore wind projects in \
Europe, including our largest renewable energy project to date.\
As of December 2021, we had enabled more than 3.5 gigawatts of \
renewable energy in Europe through 80 projects, making Amazon \
the largest purchaser of renewable energy in Europe."""

print(send_prompt(prompt_data, temperature=0.0))

Nice. We were able to extract that information and return in the answer.

Let's try another example, this time without a question that doesn't need an input text. An explicit input may not be necessary for some questions. For example, we can ask some general questions like below.

__How many months are there in a year?__

In [ ]:
prompt_data = """How many months are there in a year?"""

print(send_prompt(prompt_data, temperature=0.0))

__How many meters are in a mile?__

In [ ]:
prompt_data = """How many meters are in a mile?"""

print(send_prompt(prompt_data, temperature=0.0))

__What is the result when you add up 2 and 9?__

In [ ]:
prompt_data = """What is the result when you add up 2 and 9?"""

print(send_prompt(prompt_data, temperature=0.0))

The answers above are correct.

---

### <a name="2.3">3) Text Generation</a>
(<a href="#0">Go to top</a>)

Text generation is one of the common use cases for Large Language Models. The main purpose is to generate some high quality text considering a given input. We will cover a few examples here.

__Customer service example:__

Let's start with a customer feedback example. Assume we want to write an email to a customer who had some problems with a product that they purchased.

__Write an email response from Example Corp company customer service \
based on the following email that was received from a customer.__

__Customer email: "I am not happy with this product. I had a difficult \
time setting it up correctly because the instructions do not cover all \
the details. Even after the correct setup, it stopped working after \
a week."__

In [ ]:
prompt_data = """Write an email response from Example Corp company customer service \
based on the following email that was received from a customer.

Customer email: "I am not happy with this product. I had a difficult \
time setting it up correctly because the instructions do not cover all \
the details. Even after the correct setup, it stopped working after \
a week." """

print(send_prompt(prompt_data, temperature=0.0))

Nice! The generated text asks customer to provide more details to resolve the issue.

__Generating product descriptions:__

We can use generative AI to write creative product descriptions for our products. In the example below, we create three product descriptions for a sunglasses product.

__Product: Sunglasses.  \
Keywords: polarized, style, comfort, UV protection. \
List three variations of a detailed product \
description for the product listed above, each \
variation of the product description must \
use at least two of the listed keywords.__

In [ ]:
prompt_data = """Product: Sunglasses.  \
Keywords: polarized, style, comfort, UV protection. \
List three different product descriptions \
for the product listed above using \
at least two of the provided keywords."""

print(send_prompt(prompt_data, temperature=0.0))

---

### <a name="2.4">4) In-context learning</a>
(<a href="#0">Go to top</a>)

As pre-trained large language models learn from large and diverse data sources, they tend to build a holistic view of languages and text. This advantage allows them to learn from some input-output pairs present within the input texts. 

In this section, we will explain this __"in-context"__ learning capability with some examples. Depending on the level of information presented to the model, we can use zero-shot, one-shot or few-shot learning. We start with the most extreme case, no information presented to the model. This is called __"zero-shot-learning"__.

#### Zero-shot learning:
Assume the model is given a translation task and an input word.

__Translate English to Spanish \
 cat ==>__

In [ ]:
prompt_data = """Translate the following word from English to Spanish \
word: cat \
translation: """

print(send_prompt(prompt_data, temperature=0.0))

Correctly translated to Spanish. Let's try something different in the next one.

#### One-shot learning:
We can give the model one example and let it learn from the example to solve a problem. Below, we provide an example sentence about a cat and the model completes the second sentence about a table in a similar way.

__Answer the last question \
question: what is a cat? \
answer: cat is an animal \
\##  \
last question: what is a car?\
answer: car is__

In [ ]:
prompt_data = """Answer the last question \
question: what is a cat? \
answer: cat is an animal \
## \
last question: what is a car? \
answer: car is """

print(send_prompt(prompt_data, temperature=0.0))

It worked very well. 

#### Few-shot learning:
We can give the model multiple examples to learn from. Providing more examples can help the model produce more accurate results. Let's also change the style of the example answers by adding some __negation__ to them.

__Answer the last question \
question: what is a car? \
answer: car is not an animal \
\## \
question: what is a cat? \
answer: cat is not a vehicle \
\## \
last question: what is a shoe? \
answer: shoe is__

In [ ]:
prompt_data = """Answer the last question
question: what is a car?
answer: car is not an animal
##
question: what is a cat?
answer: cat is not a vehicle
##
last question: what is a shoe?
answer: shoe is """

print(send_prompt(prompt_data, temperature=0.0))

The response picked up the overall style very well. See that it responded starting with "not".

We can increase the __temperature__ to get different responses. Let's try that below.
Try running the cell below multiple times to obtain different answers

In [ ]:
prompt_data = """Answer the last question 
question: what is a car?
answer: car is not an animal
##
question: what is a cat?
answer: cat is not a vehicle
##
last question: what is a shoe?
answer: shoe is """

print(send_prompt(prompt_data, top_p=1.0, temperature=0.85))

Let's try one more example. This time we remove the instruction and try to complete the last sentence.

__question: what is a cat? \
answer: cat is a domesticated wild animal that belongs to the Felidae family. \
\##  \
question: what is a car? \
answer: car is a vehicle with wheels that is used for transportation. \
\##  \
last question: what is a shoe?\
answer: shoe is__

In [ ]:
prompt_data = """ 
question: what is a cat?
answer: cat is a domesticated wild animal that belongs to the Felidae family.
##
question: what is a car?
answer: car is a vehicle with wheels that is used for transportation.
##
question: what is a shoe?
answer: shoe is """

print(send_prompt(prompt_data, temperature=0.0))

It worked again. The model nicely followed the provided pattern.


---

### <a name="2.5">5) Chain of thought concept</a>
(<a href="#0">Go to top</a>)

Chain of thought concept breaks down a problem into a series of intermediate reasoning steps. This way of thinking has significantly improved the quality of the outputs generated by the Large Language Models. 

Here is the question.

### Example without Chain-of-thought (CoT)

__Answer the following question.__

__Question: When I was 16, my sister was half of my age.__ \
__Now, I’m 42. How old is my sister now?__ \

__Answer:__

In [ ]:
prompt_data = """Answer the following question.

Question: When I was 16, my sister was half of my age. \
Now, I’m 42. How old is my sister now?

Answer: """

print(send_prompt(prompt_data, temperature=0.0))

The answer is __incorrect__! This is not a big surprise. Many Large Language Models make these types of mistakes. In this case, the model skipped a few steps to solve the problem.

### Example with zero-shot CoT
It is also possible to elicit reasoning from LLMs in a **zero-shot** situation without needing to provide one-shot examples. An explicit instruction for the model to ”**think step by step**“ might help the LLM find the right solution. This approach is called zero-shot chain of thought.

__Answer the following question.__

__Question: When I was 16, my sister was half of my age.__ \
__Now, I’m 42. How old is my sister now?__ \

__Let's think step by step and describe all steps.__ \

__Answer:__

In [ ]:
prompt_data = """Answer the following question. 

Question: When I was 16, my sister was half of my age. \
Now, I’m 42. How old is my sister now?

Let's think step by step and describe all steps.

Answer: """

print(send_prompt(prompt_data, temperature=0.0))

The model answers correctly! You can also choose to provide the model with examples to guide its reasoning approach. This can enable the model to adopt different and potentially more robust reasoning strategies based on the examples provided.

### One-shot CoT
Allowing the model to **think step-by-step** allowed the model to reason through the problem and come to the correct answer.
Let's try another idea. As we have seen in the __in-context__ learning topic, LLMs tend to learn from the provided inputs and apply those learnings to another problems. Here, we will first provide the step-by-step solution for the problem with different numbers and then ask the model to solve the original problem. Since we are showing the model to reason by showing it one example, this approach is called one-shot chain of thought

__Answer the following question:__ 

__Question: When I was 10, my sister was half of my age.__ \
__Now, I’m 70. How old is my sister now?__

__Answer: When I was 10 years old, my sister was half of my age.__ \
__So, the age of the sister at that time = 10/2 = 5__ \
__This implies that the sister is 5 years younger.__ \
__Now, when I’m 70 years and age of sister = 70 - 5__ \
__Age of sister = 65.__

__Question: When I was 16, my sister was half of my age.__ \
__Now I’m 42. How old is my sister now?__

__Answer:__

In [ ]:
prompt_data = """Answer the following question.

Question: When I was 10, my sister was half of my age. \
Now, I’m 70. How old is my sister now?

Answer: When I was 10 years old, my sister was half of my age. \
So, the age of the sister at that time = 10/2 = 5 \
This implies that the sister is 5 years younger. \
Now, when I’m 70 years and age of sister = 70 - 5 \
Age of sister = 65. \

Question: When I was 16, my sister was half of my age. \
Now I’m 42. How old is my sister now?

Answer: """

print(send_prompt(prompt_data, temperature=0.0))

The model followed the given example and applied the same steps to solve the problem.

---
### <a name="3">Quiz Questions</a>
(<a href="#0">Go to top</a>)

Well done on completing the lab! Now, it's time for a brief knowledge assessment.

<div style="border: 4px solid coral; text-align: center; margin: auto;">
    <h2><i>Try it Yourself!</i></h2>
    <br>
    <p style="text-align:center;margin:auto;"><img src="../mlu_utils/challenge.png" alt="Challenge" width="100" /> </p>
    <p style=" text-align: center; margin: auto;">Answer the following questions to test your understanding of basic prompt engineering practices.</p>
    <br>
</div>

In [ ]:
import sys
sys.path.append('..')
from mlu_utils.quiz_questions import *

lab3_question1

In [ ]:
lab3_question2

# Thank you!

<p style="padding: 10px; border: 1px solid black;">
<img src="../mlu_utils/MLU-NEW-logo.png" alt="drawing" width="400"/> <br/>